In [2]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/chaulagainrupesh1/end-to-end-ml-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "chaulagainrupesh1"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "c86602b44127e4fdb1576928c0aa2ddbf3118d8d"

In [3]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\student-performance-prediction\\experiment'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\Lenovo\\Desktop\\student-performance-prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/chaulagainrupesh1/end-to-end-ml-project.mlflow"
        )
        return model_evaluation_config

In [9]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        dagshub.init(
            repo_owner='chaulagainrupesh1',
            repo_name='end-to-end-ml-project',
            mlflow=True
        )

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        # Predict
        predicted = model.predict(test_x)
        rmse, mae, r2 = self.eval_metrics(test_y, predicted)
        
        # Save metrics locally
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
        
        print("Evaluation metrics logged to MLflow and saved locally.")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-12-02 17:41:59,702: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-02 17:41:59,702: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-02 17:41:59,702: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-02 17:41:59,702: INFO: common: created directory at: artifacts]
[2025-12-02 17:41:59,714: INFO: common: created directory at: artifacts/model_evaluation]


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=c8cb46d2-7777-411f-ba99-c1eaab1ac7a8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6692ec2a8fd27e70b09d13499e0a1ad7908f300afa485bad8b3a7b7c7265e266




c:\Users\Lenovo\Desktop\student-performance-prediction\venv\Lib\site-packages\rich\live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[2025-12-02 17:42:33,876: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


...

[2025-12-02 17:42:36,683: INFO: helpers: Accessing as chaulagainrupesh1]
[2025-12-02 17:42:37,719: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/chaulagainrupesh1/end-to-end-ml-project "HTTP/1.1 200 OK"]
[2025-12-02 17:42:38,797: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "chaulagainrupesh1/end-to-end-ml-project"

[2025-12-02 17:42:38,800: INFO: helpers: Initialized MLflow to track repo "chaulagainrupesh1/end-to-end-ml-project"]


Repository chaulagainrupesh1/end-to-end-ml-project initialized!

[2025-12-02 17:42:38,803: INFO: helpers: Repository chaulagainrupesh1/end-to-end-ml-project initialized!]
[2025-12-02 17:42:38,946: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run upset-shad-611 at: https://dagshub.com/chaulagainrupesh1/end-to-end-ml-project.mlflow/#/experiments/0/runs/9de6f155014646c5bcc372b5f7ebef4c
🧪 View experiment at: https://dagshub.com/chaulagainrupesh1/end-to-end-ml-project.mlflow/#/experiments/0
Evaluation metrics logged to MLflow and saved locally.
